Mid term Report title


In [1]:
#import needed scripts
import tweepy
import json

In [2]:

twitterCredentialsPath = "TwitterCredentials.json"

with open(twitterCredentialsPath, "r") as read_file:
    credentials = json.load(read_file)

In [29]:
#authentication
auth = tweepy.Client(
	credentials["api_key"], credentials["api_key_secret"],
	credentials["consumer_key"], credentials["consumer_secret"], type(dict))
api = tweepy.API(auth)

In [30]:
auth.get_me()

AttributeError: 'Client' object has no attribute 'get_me'

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(credentials["api_key"], credentials["api_key_secret"])

auth.set_access_token(credentials["consumer_key"], credentials["consumer_secret"])

api = tweepy.API(auth)

public_tweets = api.get_users_mentions("code")

AttributeError: 'API' object has no attribute 'get_users_mentions'

In [32]:
import requests
r =requests.get("https://api.twitter.com/2/users/by?usernames=code&user.fields=id", headers={"Authorization":"Bearer "+credentials["bearer_token"]})
print(r.status_code)

401
